In [27]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df = pd.read_csv('final_ticketmaster_dataset.csv')

## GENERAL

- How many events do we have?
- are event ID's unique?
- Missingness?

In [28]:
number_of_unique_event_ids = df['event id'].nunique()
number_of_rows = df.shape[0]
number_of_unique_events = df['event name'].nunique()

if number_of_unique_event_ids == number_of_rows:
    print('The data set has one row per event id')
else:
    print('The data set has', number_of_rows, 'rows but only',  number_of_unique_event_ids, 'unique event ids')

pd.set_option('display.max_rows',120)

miss = pd.DataFrame(df.isna().sum().sort_values(ascending=False), columns = ['missing'])
miss['%'] = miss['missing']/df.shape[0]
miss

The data set has 40 rows but only 30 unique event ids


,missing,%
VIP Packages Onsale endDateTime,39,0.975
Promoter Presale endDateTime,39,0.975
Cat's Cradle Presale endDateTime,39,0.975
VIP Package Presale startDateTime,39,0.975
VIP Package Presale endDateTime,39,0.975
VIP Package Onsale startDateTime,39,0.975
VIP Package Onsale endDateTime,39,0.975
Patreon Presale startDateTime,39,0.975
Ticketmaster/Radio Presale startDateTime,39,0.975
Ticketmaster/Radio Presale endDateTime,39,0.975


In [30]:
df.groupby(['event id','event name']).count()

,,Unnamed: 0,is this entry a test?,venue name,info,public sales startDateTime,public sales startTBD,public sales startTBA,public sales endDateTime,Artist Presale startDateTime,Artist Presale endDateTime,...,event promoter 1 description,event promoter 2 name,event promoter 2 description,event promoter 3 name,event promoter 3 description,price type,price min,price max,ticket limit,pleaseNote
event id,event name,,,,,,,,,,,,,,,,,,,,,
G5eVZ4MKLbW6A,Chris Botti,2,2,2,0,2,2,2,2,0,0,...,2,2,2,2,2,2,2,2,2,2
G5eVZ4SEte6VY,Steve Hackett - Genesis Revisited: Selling England By The Pound,2,2,2,2,2,2,2,2,0,0,...,2,2,2,2,2,2,2,2,2,2
G5eVZ4URA0643,An Evening With Amy Grant,2,2,2,0,2,2,2,2,0,0,...,2,2,2,2,2,2,2,2,2,2
G5eVZ4URSmarM,Welcome To Night Vale,2,2,2,2,2,2,2,2,0,0,...,2,2,2,2,2,2,2,2,2,2
G5eVZ4UbgOhQH,Cody Ko & Noel Miller: Tiny Meat Gang - Global Domination,1,1,1,1,1,1,1,1,0,0,...,1,0,0,0,0,1,1,1,1,1
G5eVZ4UktRp32,Riverdance (Touring),1,1,1,1,1,1,1,1,0,0,...,1,0,0,0,0,1,1,1,1,1
G5eVZ4UktYU3Q,Riverdance (Touring),1,1,1,1,1,1,1,1,0,0,...,1,0,0,0,0,1,1,1,1,1
G5eVZ4UktcJ3e,Riverdance (Touring),1,1,1,1,1,1,1,1,0,0,...,1,0,0,0,0,1,1,1,1,1
G5eVZ4VTganvj,Gabriel Iglesias - Beyond The Fluffy World Tour - Go Big Or Go Home,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,1,1,1,1,1
